In [ ]:
# Imports
import numpy as np
import spikeextractors as se

np.random.seed(0)

In [ ]:
# Properties of the in-memory dataset
num_channels=7
samplerate=30000
duration=20
num_timepoints=int(samplerate*duration)
num_units=5
num_events=20

In [ ]:
# Generate a pure-noise timeseries dataset and a linear geometry
timeseries=np.random.normal(0,10,(num_channels,num_timepoints))
geom=np.zeros((num_channels,2))
geom[:,0]=range(num_channels)

# Define the in-memory recording extractor
RX=se.NumpyRecordingExtractor(timeseries=timeseries,geom=geom,samplerate=samplerate)

# Generate some random events
times=np.int_(np.sort(np.random.uniform(0,num_timepoints,num_events)))
labels=np.random.randint(1,num_units+1,size=num_events)
    
# Define the in-memory sorting extractor
SX=se.NumpySortingExtractor()
for k in range(1,num_units+1):
    times_k=times[np.where(labels==k)[0]]
    SX.add_unit(unit_id=k,times=times_k)
    
#Add some features to the sorting extractor. These will be merged and split appropriately during curation
spikes = 0
for unit_id in SX.get_unit_ids():
    SX.set_unit_spike_features(unit_id, feature_name='f_int', value=range(spikes, spikes + len(SX.get_unit_spike_train(unit_id))))
    spikes += len(SX.get_unit_spike_train(unit_id))
    
spikes = 0
for unit_id in SX.get_unit_ids():
    SX.set_unit_spike_features(unit_id, feature_name='f_float', value=np.arange(float(spikes) + .1, float(spikes + len(SX.get_unit_spike_train(unit_id) + .1))))
    spikes += len(SX.get_unit_spike_train(unit_id))
    
#Features that are not shared across ALL units will not be merged and split correctly (will disappear)
SX.set_unit_spike_features(1, feature_name='bad_feature', value=np.repeat(1, len(SX.get_unit_spike_train(1))))
SX.set_unit_spike_features(2, feature_name='bad_feature', value=np.repeat(2, len(SX.get_unit_spike_train(2))))
SX.set_unit_spike_features(3, feature_name='bad_feature', value=np.repeat(3, len(SX.get_unit_spike_train(3))))

In [ ]:
# Demonstrate the API for extracting information
print('Unit ids = {}'.format(SX.get_unit_ids()))
st=SX.get_unit_spike_train(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))

In [ ]:
# Now we can curate the results using a CuratedSortingExtractor
CSX = se.CuratedSortingExtractor(parent_sorting=SX)

In [ ]:
print("Curated Unit Ids: " + str(CSX.get_unit_ids()))
print("Original Unit Ids: " + str(SX.get_unit_ids()))

print("Curated ST: " + str(CSX.get_unit_spike_train(1)))
print("Original ST: " + str(SX.get_unit_spike_train(1)))

In [ ]:
#Lets split one unit from the sorting result (this could be two units incorrectly clustered as one)
CSX.split_unit(unit_id=1, indices=[0, 1])
print("Curated Unit Ids: " + str(CSX.get_unit_ids()))
print("Original Spike Train: " + str(SX.get_unit_spike_train(1)))
print("Split Spike Train 1: " + str(CSX.get_unit_spike_train(6)))
print("Split Spike Train 2: " + str(CSX.get_unit_spike_train(7)))
for unit_id in CSX.get_unit_ids():
    CSX.printCurationTree(unit_id=unit_id)

In [ ]:
# If the split was incorrect, we can always merge the two units back together
CSX.merge_units(unit_ids=[6, 7])
print("Curated Spike Train: " + str(CSX.get_unit_spike_train(8)))
print("Original Spike Train: " + str(SX.get_unit_spike_train(1)))
for unit_id in CSX.get_unit_ids():
    CSX.printCurationTree(unit_id=unit_id)

In [ ]:
# We can also exclude units, so let's get rid of 8 since we are just confused about this unit
CSX.exclude_units(unit_ids=[8])
for unit_id in CSX.get_unit_ids():
    CSX.printCurationTree(unit_id=unit_id)

In [ ]:
#Now let's merge 3 and 4 together (This will create a new unit which encapsulates both previous units)
CSX.merge_units(unit_ids=[3, 4])
print("Curated Unit Ids: " + str(CSX.get_unit_ids()))
print("Merged Spike Train: " + str(CSX.get_unit_spike_train(9)))
print("Original Spike Trains concatenated: " + str(np.sort(np.concatenate((SX.get_unit_spike_train(3), SX.get_unit_spike_train(4))))))
print("\nCuration Tree")
for unit_id in CSX.get_unit_ids():
    CSX.printCurationTree(unit_id=unit_id)

In [ ]:
#Now let's merge 2 and 6 together

CSX.merge_units(unit_ids=[2, 9])
print("Curated Unit Ids: " + str(CSX.get_unit_ids()))
print("Merged Spike Train: " + str(CSX.get_unit_spike_train(10)))
merged_spike_train = []
for unit_id in SX.get_unit_ids():
    if(unit_id != 1 and unit_id != 5):
        merged_spike_train.append(SX.get_unit_spike_train(unit_id))
merged_spike_train = np.asarray(merged_spike_train)
merged_spike_train = np.sort(np.concatenate(merged_spike_train).ravel())
print("Original Spike Trains concatenated: " + str(merged_spike_train))
print("\nCuration Tree")
for unit_id in CSX.get_unit_ids():
    CSX.printCurationTree(unit_id=unit_id)

In [ ]:
#Now let's split unit 5 with given indices

CSX.split_unit(unit_id=5, indices=[0, 1])
print("Curated Unit Ids: " + str(CSX.get_unit_ids()))
print("Original Spike Train: " + str(SX.get_unit_spike_train(5)))
print("Split Spike Train 1: " + str(CSX.get_unit_spike_train(11)))
print("Split Spike Train 2: " + str(CSX.get_unit_spike_train(12)))
print("\nCuration Tree")
for unit_id in CSX.get_unit_ids():
    CSX.printCurationTree(unit_id=unit_id)

In [ ]:
#Finally, we can merge units 7 and 8

CSX.merge_units(unit_ids=[10, 11])
print("Curated Unit Ids: " + str(CSX.get_unit_ids()))
print("Merged Spike Train: " + str(CSX.get_unit_spike_train(13)))
original_spike_train = (np.sort(np.concatenate((SX.get_unit_spike_train(3), SX.get_unit_spike_train(4), SX.get_unit_spike_train(2), SX.get_unit_spike_train(5)[np.asarray([0,1])]))))
print("Original Spike Train: " + str(original_spike_train))
print("\nCuration Tree")
for unit_id in CSX.get_unit_ids():
    CSX.printCurationTree(unit_id=unit_id)

In [ ]:
# Write the input/output in the MountainSort format
se.MdaRecordingExtractor.write_recording(recording=RX,save_path='sample_mountainsort_dataset')
se.MdaSortingExtractor.write_sorting(sorting=CSX,save_path='sample_mountainsort_dataset/firings_true.mda')

In [ ]:
# Read this dataset with the Mda input extractor (we can now have a normal sorting extractor with our curations)
RX2=se.MdaRecordingExtractor(dataset_directory='sample_mountainsort_dataset')
SX2=se.MdaSortingExtractor(firings_file='sample_mountainsort_dataset/firings_true.mda')

In [ ]:
print("New Unit Ids: " + str(SX2.get_unit_ids()))
print("New Unit Spike Train: " + str(SX2.get_unit_spike_train(13)))
print("Previous Curated Unit Spike Train: " + str(CSX.get_unit_spike_train(13)))

In [ ]:
#Current spike feature names
print(CSX.get_unit_spike_feature_names())

In [ ]:
#All features have been appropriately merged and split according to previous operations
print(CSX.get_unit_spike_features(12, 'f_int'))
print(CSX.get_unit_spike_features(12, 'f_float'))
print(CSX.get_unit_spike_features(13, 'f_int'))
print(CSX.get_unit_spike_features(13, 'f_float'))